# Strategy Report Notebook
This notebok runs analysis on a given strategy.

## Import Libraries

In [ ]:
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
from xbbg import blp


## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for Equity Benchmark

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**):  SPTR, M1WD, SX5T

In [ ]:
equity_bmk = 'SPTR'
# strat_drop_list = ['99%/90% Put Spread','Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR',
#        'GW Dispersion', 'Corr Hedge', 'Def Var']
# include_fi = False
#create returns data dictionary for equity benchmark
sptr_dict= dm.get_equity_hedge_returns(equity_bmk, only_equity=True)

In [ ]:
jpm_comm_list = ['JMABAB3E Index','JMABDBSE Index','JMABSEAS Index','JMABNEO1 Index','JMABNCMF Index',
                 'JMABCIDR Index','JMABCID2 Index','JCOPCF Index','JCOPCF2 Index',
                 'JMABSSPE Index','JMABMCV1 Index','JMABNEO2 Index','JMABJH01 Index',
                 'JMABNCC1 Index','JMABNCL1 Index','JMABNCM1 Index','JMABNCV1 Index']
jpm_comd_uni = blp.bdh(tickers=jpm_comm_list, flds='PX_LAST', start_date='2007-12-31', end_date='2021-08-31', Per='D', Fill='P', Days='T')
# .get_data(jpm_comm_list, start_date='2007-12-31', end_date='2021-08-31')
jpm_comd_uni.drop(['JMABMCV1 Index'], axis = 1, inplace=True)
jpm_comd_uni.dropna(inplace=True)
jpm_comd_uni.columns = ['JMABDBSE Index', 'JMABSEAS Index', 'JMABNEO1 Index', 'JMABNCMF Index', 'JMABCIDR Index', 'JMABCID2 Index', 'JCOPCF Index',
 'JCOPCF2 Index', 'JMABSSPE Index', 'JMABNCC1 Index', 'JMABNCL1 Index', 'JMABNCM1 Index', 'JMABNCV1 Index']
jpm_comd_uni.head()

In [ ]:
# list(jpm_comd_uni.columns)
# ['JMABDBSE Index', 'JMABSEAS Index', 'JMABNEO1 Index', 'JMABNCMF Index', 'JMABCIDR Index', 'JMABCID2 Index', 'JCOPCF Index',
#  'JCOPCF2 Index', 'JMABSSPE Index', 'JMABNCC1 Index', 'JMABNCL1 Index', 'JMABNCM1 Index', 'JMABNCV1 Index']

## Import Daily, Weekly, Monthly, Quarterly and Yearly Returns Data for a Strategy
To import the returns data:
* In **new_data = get_new_strategy_returns_data( report_name= , sheet_name = )**
    * Add File Name to **report_name** 
    * Add sheet name to **sheet_name**

In [ ]:
# ubs_vortex= dm.get_new_strategy_returns_data( dm.NEW_DATA+'Convixity Timeseries_8.5.21.xlsx'
#                                            sheet_name = "Series")
# barclays_vortex= dm.get_new_strategy_returns_data( dm.NEW_DATA+'Barclays_Volatility_Convexity_New_Iterations_08042021.xlsx'
#                                            sheet_name = "New Iterations")

#create returns data dictionary for strategy
new_data_dict = dm.get_data_dict(jpm_comd_uni)

#merge dictionaries
ret_data_dict = dm.merge_dicts(sptr_dict,new_data_dict)

## View Returns Data
Select frequency (Daily, Weekly, Monthly, Yearly) to view data

In [ ]:
freq_list = ['Daily', 'Weekly', 'Monthly','Quarterly', 'Yearly']
@interact
def display_returns(frequency = freq_list):
    return fmt.get_returns_styler(ret_data_dict[frequency])

# Compute Correlations

In [ ]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict = summary.get_corr_data(ret_data_dict)

# Display Correlations

Display Correlations of returns data by:
* **frequency (Monthly or Weekly)** - Show correlations of Monthly or Weekly returns
* **corr** - Show:
    * **corr** - full history correaltions
    * **corr_down** - risk seeking benchmark downside returns correlations or 
    * **corr_up** - risk seeking benchmark upside returns correlations
* **plot_type (corrplot or heatmap)** - Show a map with magnitude or just heatmap

In [ ]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['corr', 'corr_down', 'corr_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

## Compute Return Stats and hedge Metrics

In [ ]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(ret_data_dict)

## Display Return Stats and Hedge Metrics
Display Returns Analytics data by:
* **frequency (Monthly or Weekly)** - Show Monthly or Weekly returns
* **stats (return_stats, hedge_metrics)** - Show:
    * Return Statistics or 
    * Hedge Metrics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs

In [ ]:
hist_dict = summary.get_hist_data(ret_data_dict)

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs():
    df_hist = hist_dict[False]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis
Display grouped data by:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles

In [ ]:
group_list=['Quintile','Decile']
@interact
def display_returns(group = group_list):
    quintile_df = summary.get_grouped_data(ret_data_dict, group = group)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile or Decile Analysis
Display Quintile Analysis Bar Chart:
* **group (Quintile or Decile)** - Show returns data in quintiles or deciles
* **strat** - Show quintile or decile for selected strategy

In [ ]:
group_list=['Quintile','Decile']
@interact

def display_quintile_analysis(group = group_list, strat=list(ret_data_dict['Monthly'].select_dtypes('float').columns)[1:]):    
    df = summary.get_grouped_data(ret_data_dict, group = group)
    
    for col in df.columns:
        if not (col == equity_bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.show()
    

# Create Report

Run this code below to export analysis into excel spreadsheet
* **strat_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [ ]:
strat_report = 'Vortex_analysis_08_16_2021'
selloffs = True
rp.generate_strat_report(strat_report, ret_data_dict, selloffs)